In [22]:
!pip install ibm-cos-sdk
!pip install matplotlib
!pip install pandas
!pip install scikit-learn


[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [23]:
# Enable interactive matplotlib plots
%matplotlib notebook

2024-01-09 09:40:42,743 matplotlib [DEBUG] matplotlib data path: /opt/app-root/lib64/python3.9/site-packages/matplotlib/mpl-data
2024-01-09 09:40:42,743 matplotlib [DEBUG] matplotlib data path: /opt/app-root/lib64/python3.9/site-packages/matplotlib/mpl-data
2024-01-09 09:40:42,743 matplotlib [DEBUG] matplotlib data path: /opt/app-root/lib64/python3.9/site-packages/matplotlib/mpl-data
2024-01-09 09:40:42,750 matplotlib [DEBUG] CONFIGDIR=/opt/app-root/src/.config/matplotlib
2024-01-09 09:40:42,750 matplotlib [DEBUG] CONFIGDIR=/opt/app-root/src/.config/matplotlib
2024-01-09 09:40:42,750 matplotlib [DEBUG] CONFIGDIR=/opt/app-root/src/.config/matplotlib
2024-01-09 09:40:42,754 matplotlib [DEBUG] interactive is False
2024-01-09 09:40:42,754 matplotlib [DEBUG] interactive is False
2024-01-09 09:40:42,754 matplotlib [DEBUG] interactive is False
2024-01-09 09:40:42,756 matplotlib [DEBUG] platform is linux
2024-01-09 09:40:42,756 matplotlib [DEBUG] platform is linux
2024-01-09 09:40:42,756 matpl

In [41]:
import logging
ibm_boto3.set_stream_logger('', logging.WARNING)

In [ ]:
import os
import ibm_boto3


# Constants for IBM COS values
COS_ENDPOINT = os.getenv('COS_ENDPOINT') # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
COS_API_KEY_ID = os.getenv('COS_API_KEY_ID') # eg "W00YixxxxxxxxxxMB-odB-2ySfTrFBIQQWanc--P3byk"
COS_INSTANCE_CRN = os.getenv('COS_INSTANCE_CRN') # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003xxxxxxxxxx1c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_AUTH_ENDPOINT = os.getenv('COS_AUTH_ENDPOINT') #eg "https://iam.cloud.ibm.com/identity/token"
REGION = os.getenv('REGION')

# Create resource
cos = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT,
    region_name="eu-de"
)

dataset_path = 'ceiling-fan-dataset'  # Directory where raw accelerometer data is stored
normal_op_list = ['fan_0_low-deploy']
anomaly_op_list = ['fan_0_med_0_weight', 'fan_0_high_0_weight',
                  'fan_0_low_1_weight', 'fan_0_med_1_weight', 'fan_0_high_1_weight']

total_list = normal_op_list + anomaly_op_list
if not os.path.exists('./' + dataset_path):
    for folder in total_list:
        os.makedirs(dataset_path + '/' + folder)


bucket = 'tead-bucket'

files = cos.Bucket(bucket).objects.all()
for file in files:
    #ceiling-fan/fan_0_low_0_weight/0171.csv
    sample = file.key.split("/")
    if len(sample) == 3:
        if sample[2]:
            if sample[1] in total_list:
                local_file_name = file.key.replace("ceiling-fan", "ceiling-fan-dataset")
                try:
                    cos.meta.client.download_file(bucket, file.key, local_file_name)
                except Exception as e:
                    print(Exception, e)
                else:
                    print("Item: {0} ({1} bytes).".format(file.key, file.size))

Item: ceiling-fan/fan_0_high_0_weight/0000.csv (5838 bytes 3rd string 0000.csv).
Item: ceiling-fan/fan_0_high_0_weight/0001.csv (5826 bytes 3rd string 0001.csv).
Item: ceiling-fan/fan_0_high_0_weight/0002.csv (5842 bytes 3rd string 0002.csv).
Item: ceiling-fan/fan_0_high_0_weight/0003.csv (5819 bytes 3rd string 0003.csv).
Item: ceiling-fan/fan_0_high_0_weight/0004.csv (5831 bytes 3rd string 0004.csv).
Item: ceiling-fan/fan_0_high_0_weight/0005.csv (5827 bytes 3rd string 0005.csv).
Item: ceiling-fan/fan_0_high_0_weight/0006.csv (5837 bytes 3rd string 0006.csv).
Item: ceiling-fan/fan_0_high_0_weight/0007.csv (5838 bytes 3rd string 0007.csv).
Item: ceiling-fan/fan_0_high_0_weight/0008.csv (5857 bytes 3rd string 0008.csv).
Item: ceiling-fan/fan_0_high_0_weight/0009.csv (5829 bytes 3rd string 0009.csv).
Item: ceiling-fan/fan_0_high_0_weight/0010.csv (5838 bytes 3rd string 0010.csv).
Item: ceiling-fan/fan_0_high_0_weight/0011.csv (5818 bytes 3rd string 0011.csv).
Item: ceiling-fan/fan_0_high